# Scenario Comparison

## Overview
- **What**: Side-by-side scenario comparison framework with automatic insight extraction, smart annotation placement, A/B testing visualizations, and natural-language executive summaries.
- **Prerequisites**: [01_visualization_factory](01_visualization_factory.ipynb), [../core/04_monte_carlo_engine](../core/04_monte_carlo_engine.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Practitioner] / [Developer]

## Topics Covered
1. Creating scenarios via `ScenarioManager`
2. Comparing scenarios with `ScenarioComparator`
3. Automatic insight extraction
4. Smart annotation placement
5. A/B testing visualization
6. Exporting comparison reports

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/visualization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

from ergodic_insurance.config_manager import ConfigManager
from ergodic_insurance.scenario_manager import ScenarioManager, ParameterSpec
from ergodic_insurance.reporting.scenario_comparator import ScenarioComparator
from ergodic_insurance.reporting.insight_extractor import InsightExtractor
from ergodic_insurance.visualization.annotations import (
    SmartAnnotationPlacer,
    auto_annotate_peaks_valleys,
    create_leader_line,
)
from ergodic_insurance.visualization.core import set_wsj_style

set_wsj_style()
np.random.seed(42)

print("Scenario comparison framework loaded.")

## 1. Create Scenarios

Define parameter specifications and generate a sensitivity analysis grid from a base configuration.

In [ ]:
scenario_mgr = ScenarioManager()
config_mgr = ConfigManager()
base_config = config_mgr.load_profile("default")

param_specs = [
    ParameterSpec(name="insurance.base_premium_rate", base_value=0.015, variation_pct=0.3),
    ParameterSpec(name="insurance.limit", base_value=5_000_000, variation_pct=0.5),
    ParameterSpec(name="manufacturer.target_margin", base_value=0.08, variation_pct=0.25),
]

scenarios = scenario_mgr.create_sensitivity_analysis(
    base_name="insurance_optimization",
    parameter_specs=param_specs,
    base_config=base_config,
    tags={"demo", "comparison"},
)

print(f"Created {len(scenarios)} scenarios:")
for s in scenarios:
    print(f"  - {s.name}: {s.description}")

## 2. Generate Results

In production these would come from actual simulations. Here we create synthetic results that respond realistically to parameter changes.

In [ ]:
def create_synthetic_results(scenario_name, param_overrides):
    np.random.seed(hash(scenario_name) % 2**32)
    base_growth, base_ruin, base_assets = 0.06, 0.015, 10_000_000

    if "insurance.base_premium_rate" in param_overrides:
        eff = (param_overrides["insurance.base_premium_rate"] - 0.015) * 100
        base_growth -= eff * 0.5
        base_ruin += eff * 0.3
    if "insurance.limit" in param_overrides:
        eff = (param_overrides["insurance.limit"] - 5_000_000) / 5_000_000
        base_ruin *= 1 - eff * 0.4
        base_growth += eff * 0.01
    if "manufacturer.target_margin" in param_overrides:
        eff = (param_overrides["manufacturer.target_margin"] - 0.08) / 0.08
        base_growth += eff * 0.02
        base_assets *= 1 + eff * 0.15

    growth = base_growth + np.random.normal(0, 0.005)
    ruin = max(0, base_ruin + np.random.normal(0, 0.002))
    assets = base_assets * (1 + np.random.normal(0, 0.1))

    return {
        "summary_statistics": {
            "mean_growth_rate": growth,
            "ruin_probability": ruin,
            "mean_final_assets": assets,
            "var_95": -np.random.uniform(50_000, 200_000),
            "var_99": -np.random.uniform(100_000, 500_000),
            "sharpe_ratio": growth / 0.15,
            "max_drawdown": np.random.uniform(0.05, 0.25),
        },
        "config": param_overrides,
    }


results = {
    s.name: create_synthetic_results(s.name, s.parameter_overrides)
    for s in scenarios
}
print(f"Generated results for {len(results)} scenarios")

## 3. Compare Scenarios

The `ScenarioComparator` ranks scenarios against a baseline and identifies top performers.

In [ ]:
comparator = ScenarioComparator()

comparison = comparator.compare_scenarios(
    results,
    baseline="insurance_optimization_baseline",
    metrics=["mean_growth_rate", "ruin_probability", "mean_final_assets", "sharpe_ratio"],
)

print(f"Scenarios compared: {len(comparison.scenarios)}")

print("\nTop performers:")
for metric in ["mean_growth_rate", "ruin_probability"]:
    ascending = "ruin" in metric
    top = comparison.get_top_performers(metric, n=3, ascending=ascending)
    print(f"  {metric.replace('_', ' ').title()}:")
    for name, val in top:
        print(f"    {name}: {val:.4f}")

## 4. Comparison Grid

Bar charts for each metric with difference indicators from baseline.

In [ ]:
fig = comparator.create_comparison_grid(
    metrics=["mean_growth_rate", "ruin_probability", "mean_final_assets", "sharpe_ratio"],
    figsize=(16, 10),
    show_diff=True,
)
plt.tight_layout()
plt.show()

## 5. Insight Extraction

The `InsightExtractor` automatically identifies key findings and generates an executive summary.

In [ ]:
extractor = InsightExtractor()

insights = extractor.extract_insights(
    comparison,
    focus_metrics=["mean_growth_rate", "ruin_probability"],
    threshold_importance=50,
)

print(f"Extracted {len(insights)} insights:\n")
for i, ins in enumerate(insights[:5], 1):
    print(f"{i}. [{ins.category.upper()}] {ins.title}")
    print(f"   {ins.description}")
    print(f"   Importance: {ins.importance:.0f}/100 | Confidence: {ins.confidence:.1%}\n")

executive_summary = extractor.generate_executive_summary(max_points=5, focus_positive=True)
print("\nExecutive Summary:")
print(executive_summary)

## 6. Smart Annotations

The `SmartAnnotationPlacer` prevents overlapping labels on complex plots. Peak/valley auto-detection annotates extrema.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))

np.random.seed(42)
x = np.arange(100)
y1 = np.cumsum(np.random.randn(100) * 0.5) + 50
y2 = np.cumsum(np.random.randn(100) * 0.3) + 45
y3 = np.cumsum(np.random.randn(100) * 0.4) + 40

ax.plot(x, y1, label="Optimized Strategy", linewidth=2, color="#1f77b4")
ax.plot(x, y2, label="Baseline Strategy", linewidth=2, color="#ff7f0e")
ax.plot(x, y3, label="Conservative Strategy", linewidth=2, color="#2ca02c")

placer = SmartAnnotationPlacer(ax)
placer = auto_annotate_peaks_valleys(
    ax, x, y1, n_peaks=2, n_valleys=1,
    peak_color="#006400", valley_color="#8B0000", fontsize=8, placer=placer,
)

annotations = [
    {"text": "Optimized outperforms", "point": (60, y1[60]), "priority": 85, "color": "#4169E1"},
    {"text": "Baseline steady", "point": (35, y2[35]), "priority": 75, "color": "#FF8C00"},
    {"text": "Conservative lags", "point": (75, y3[75]), "priority": 65, "color": "#556B2F"},
]
placer.add_smart_annotations(annotations, fontsize=9)

ax.set_xlabel("Time Period", fontsize=11)
ax.set_ylabel("Performance Metric", fontsize=11)
ax.set_title("Smart Annotation Demonstration", fontsize=13, fontweight="bold", pad=20)
ax.legend(loc="lower right", framealpha=0.95, edgecolor="gray")
ax.grid(True, alpha=0.2, linestyle="--", linewidth=0.5)
ax.set_xlim(-5, 105)
y_min = min(y1.min(), y2.min(), y3.min())
y_max = max(y1.max(), y2.max(), y3.max())
pad = (y_max - y_min) * 0.15
ax.set_ylim(y_min - pad, y_max + pad)

plt.tight_layout()
plt.show()

## 7. A/B Testing Visualization

Direct comparison of two scenarios across all key metrics with improvement indicators.

In [ ]:
scenario_a = "insurance_optimization_baseline"
scenario_b = list(comparison.scenarios)[1] if len(comparison.scenarios) > 1 else scenario_a
metrics_to_compare = ["mean_growth_rate", "ruin_probability", "mean_final_assets", "sharpe_ratio"]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for idx, metric in enumerate(metrics_to_compare):
    ax = axes.flat[idx]
    val_a = comparison.metrics[metric].get(scenario_a, 0)
    val_b = comparison.metrics[metric].get(scenario_b, 0)

    bars = ax.bar(["Baseline", "Alternative"], [val_a, val_b],
                   color=["#1f77b4", "#ff7f0e"], alpha=0.8)
    for bar, val in zip(bars, [val_a, val_b]):
        if "probability" in metric or "rate" in metric:
            label = f"{val:.2%}"
        elif "assets" in metric:
            label = f"${val / 1e6:.1f}M"
        else:
            label = f"{val:.3f}"
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
               label, ha="center", va="bottom", fontweight="bold")

    diff = ((val_b - val_a) / val_a * 100) if val_a != 0 else 0
    worse_is_lower = "ruin" in metric or "drawdown" in metric
    improvement = diff < 0 if worse_is_lower else diff > 0
    color = "green" if improvement else "red"
    status = "Better" if improvement else "Worse"
    ax.text(0.5, 0.95, f"{diff:+.1f}% {status}",
           transform=ax.transAxes, ha="center", va="top",
           fontsize=12, fontweight="bold", color=color,
           bbox=dict(boxstyle="round", facecolor="white", edgecolor=color, alpha=0.8))

    ax.set_title(metric.replace("_", " ").title())
    ax.set_ylabel("Value")
    ax.grid(True, alpha=0.3, axis="y")

fig.suptitle(f"A/B Test: {scenario_a} vs {scenario_b}",
             fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

## 8. Export Comparison Report

Save the full comparison (data, plots, and insights) to disk.

In [ ]:
import os

output_dir = "comparison_reports"
os.makedirs(output_dir, exist_ok=True)

output_files = comparator.export_comparison_report(
    os.path.join(output_dir, "scenario_comparison"),
    include_plots=True,
)

insights_path = os.path.join(output_dir, "insights.md")
extractor.export_insights(insights_path, output_format="markdown")

print("Exported files:")
for key, path in output_files.items():
    print(f"  {key}: {path}")
print(f"  insights: {insights_path}")

## Key Takeaways

- `ScenarioComparator` provides automated ranking, grid plots, and difference tables.
- `InsightExtractor` generates prioritized findings and executive summaries from raw results.
- `SmartAnnotationPlacer` prevents label overlaps on complex multi-series plots.
- A/B testing visualizations make direct scenario comparisons immediately understandable.
- Export all results (CSV, plots, Markdown) for stakeholder distribution.

## Next Steps

- [../reporting/01_report_generation](../reporting/01_report_generation.ipynb) -- full report generation pipeline
- [02_executive_dashboards](02_executive_dashboards.ipynb) -- executive visualization suite
- [../optimization/01_basic_optimization](../optimization/01_basic_optimization.ipynb) -- optimization workflows